In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import math

Using TensorFlow backend.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

GeForce GTX 1050 Ti with Max-Q Design


In [3]:
from pytorch_transformers import BertTokenizer, BertConfig

In [4]:
from pytorch_transformers import AdamW, BertForSequenceClassification
from pytorch_transformers import BertForQuestionAnswering, BertForTokenClassification

In [5]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.load_state_dict(torch.load('D:/Project/Work_Project/final/data/models/BERT_model.h5'))

<All keys matched successfully>

In [6]:
import pandas as pd

texts = pd.read_csv('D:/Project/Work_Project/final/data/processed/Dataset_text_1.csv', encoding='utf8')
texts = texts.dropna(axis='index', how='any', subset=['CLASS']) 
mir_texts = texts[texts['CLASS'] == 0]
maf_texts = texts[texts['CLASS'] == 1]
sentences = np.concatenate([maf_texts['TEXT'].values, mir_texts['TEXT'].values])

sentences = ['[CLS] ' + str(sentence) + ' [SEP]' for sentence in sentences]
labels = [[1] for _ in range(maf_texts.shape[0])] + [[0] for _ in range(mir_texts.shape[0])]

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
tok = tokenizer.tokenize(sentences[0])
print (tok)

['[CLS]', 'г', '##о', '##р', '##о', '##д', 'з', '##д', '##е', '##с', '##ь', 'м', '##ы', 'с', '##е', '##и', '##ч', '##а', '##с', 'в', '##с', '##е', 'з', '##на', '##к', '##о', '##м', '##и', '##м', '##с', '##я', 'в', '##с', '##е', 'р', '##а', '##з', '##г', '##ов', '##а', '##р', '##и', '##в', '##а', '##л', '##и', 'д', '##о', '##б', '##р', '##ы', '##и', 'в', '##е', '##ч', '##е', '##р', 'с', '##п', '##а', '##с', '##и', '##б', '##о', 'с', '##п', '##а', '##с', '##и', '##б', '##о', 'и', '##г', '##р', '##о', '##к', 'н', '##о', '##м', '##е', '##р', 'о', '##д', '##и', '##н', '[SEP]']


In [8]:
input_ids = tokenizer.convert_tokens_to_ids(tok)
input_ids = pad_sequences(
    [input_ids, ''],
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)

attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [9]:
train_inputs = torch.tensor(input_ids[0]).long().to(device)
train_labels = torch.tensor(labels[0]).long().to(device)
train_masks = torch.tensor(attention_masks[0]).long().to(device)

In [10]:
train_inputs = train_inputs.unsqueeze_(0)
train_labels = train_labels.unsqueeze_(0)
train_masks = train_masks.unsqueeze_(0)


In [11]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
logits = model(train_inputs, token_type_ids=None, attention_mask=train_masks)

In [13]:
logits

(tensor([[-0.3971,  0.1098]], device='cuda:0', grad_fn=<AddmmBackward>),)